## Data Loaders

This notebook demonstrates the use of **vertex loader** in `pyTigerGraph`. The job of a data loader is to pull data from the TigerGraph database. Currently, the following data loaders are provided:
* EdgeLoader, which returns batches of edges.
* VertexLoader, which returns batches of vertices.
* GraphLoader, which returns randomly sampled (probably disconnected) subgraphs in pandas `dataframe`, `PyG` or `DGL` format.
* NeighborLoader, which returns subgraphs using neighbor sampling in `dataframe`, `PyG` or `DGL` format.
* EdgeNeighborLoader, which returns subgraphs using neighbor sampling from edges in `dataframe`, `PyG` or `DGL` format.

Every data loader above can either get all the batches as a HTTP response (default) or stream every batch through Kafka. The former mechanism is good for testing with small graphs and it is fast, but it subjects to a data size limit of 2GB. For large graphs, the HTTP channel will likely fail due to size limit and network connectivity issues. Streaming via Kafka is offered for data robustness and scalability. Also, Kafka excels at multi-consumer use cases, and it is efficient for model search or hyperparameter tuning when there are multiiple consumers of the same data. 

The data loaders support both homogeneous and heterogenous graphs. By default, they load from all vertex and edge types and treat the graph as a homogeneous graph. But they also allow users to specify what vertex and edge types to load from and what attributes to load from each type. This way users will get heterogeneous graph outputs.

**NOTE**: Currently, your database needs to be activated (only once) to enjoy all the functions provided by the ML Workbench. If you are using ML Workbench on Cloud, then the activator is included and you can run the cell below (uncomment first) to activate. For other versions of the Workbench, you can download the activator at https://act.tigergraphlabs.com. Detailed instructions are also included on that website. 

In [ ]:
# Uncomment below and fill out the necessary information. For detailed instructions, please see https://act.tigergraphlabs.com
# !mlwb activate [database address] -u [username] -p [password] -s [secret]

### Connection to Database

The `TigerGraphConnection` class represents a connection to the TigerGraph database. Under the hood, it stores the necessary information to communicate with the database. It is able to perform quite a few database tasks. Please see its [documentation](https://docs.tigergraph.com/pytigergraph/current/intro/) for details.

In [2]:
from pyTigerGraph import TigerGraphConnection

conn = TigerGraphConnection(
    host="http://127.0.0.1", # Change the address to your database server's
    graphname="Cora",
    username="tigergraph",
    password="tigergraph",
)

<span style="color:red">Uncomment cell below and run to get and set token if token authentication is enabled</span>. 
* This is required for all databases on tgcloud.
* `<secret>` is your user secret. See https://docs.tigergraph.com/tigergraph-server/current/user-access/managing-credentials#_secrets for details.
* If you don't know your secret, you can use `secret=conn.createSecret()` to create one.

In [ ]:
#conn.getToken(<secret>)

In [3]:
# Number of vertices for every vertex type
conn.getVertexCount('*')

{'Paper': 2708}

In [4]:
# Number of edges for every type
conn.getEdgeCount()

{'Cite': 10556}

### Vertex Loader

`VertexLoader` pulls batches of vertices from database. Specifically, it divides vertices into `num_batches` and returns each batch separately. The boolean attribute provided to `filter_by` indicates which vertices are included. If you need random batches, set `shuffle` to True.

**Note**: For the first time you initialize the loader on a graph in TigerGraph, the initialization might take a minute as it installs the corresponding query to the database and optimizes it. However, the query installation only needs to be done once, so it will take no time when you initialize the loader on the same TG graph again.

There are two ways to use the data loader. See [here](https://github.com/TigerGraph-DevLabs/mlworkbench-docs/blob/main/tutorials/basics/2_dataloaders.ipynb) for examples.
* First, it can be used as an iterable, which means you can loop through it to get every batch of data. If you load all vertices at once (`num_batches=1`), there will be only one batch (of all the vertices) in the iterator.
* Second, you can access the `data` property of the class directly. If there is only one batch of data to load, it will give you the batch directly instead of an iterator, which might make more sense in that case. If there are multiple batches of data to load, it will return the loader again.

Args:
* attributes (list or dict, optional):
                Vertex attributes to be included. If it is a list, then the attributes
                in the list from all vertex types will be selected. An error will be thrown if
                certain attribute doesn't exist in all vertex types. If it is a dict, keys of the 
                dict are vertex types to be selected, and values are lists of attributes to be 
                selected for each vertex type. Defaults to None.
* batch_size (int, optional):
                Number of vertices in each batch.
                Defaults to None.
* num_batches (int, optional):
                Number of batches to split the vertices.
                Defaults to 1.
* shuffle (bool, optional):
                Whether to shuffle the vertices before loading data.
                Defaults to False.
* filter_by (str, optional):
                A boolean attribute used to indicate which vertices
                can be included. Defaults to None.
* output_format (str, optional):
                Format of the output data of the loader. Only
                "dataframe" is supported. Defaults to "dataframe".
* loader_id (str, optional):
                An identifier of the loader which can be any string. It is
                also used as the Kafka topic name. If `None`, a random string will be generated
                for it. Defaults to None.
* buffer_size (int, optional):
                Number of data batches to prefetch and store in memory. Defaults to 4.
* timeout (int, optional):
                Timeout value for GSQL queries, in ms. Defaults to 300000.

#### Load all vertices at once directly to local. Default.

In [6]:
%%time
vertex_loader = conn.gds.vertexLoader(
    num_batches=1, 
    attributes=["id", "x", "y"])
# Note: an unique ID (vid) of each vertex will be automatically generated and extracted. 
# If you want the primary ID of each vertex, specify it as an attribute.

Installing and optimizing queries. It might take a minute if this is the first time you use this loader.
Query installation finished.
CPU times: user 223 ms, sys: 34.9 ms, total: 258 ms
Wall time: 1min 1s


In [7]:
%%time
# Get the only batch of data via the `data` property
data = vertex_loader.data
data.shape

CPU times: user 94.4 ms, sys: 33.9 ms, total: 128 ms
Wall time: 225 ms


(2708, 4)

In [8]:
data.head()

,vid,id,x,y
0,0,1596,0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...,2
1,1,2703,0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 ...,3
2,2,2379,0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 ...,4
3,3,1480,0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...,2
4,4,2163,0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 ...,3


#### Get batches of vertices through http

In [9]:
%%time
vertex_loader = conn.gds.vertexLoader( 
    num_batches=20, 
    attributes=["id", "x", "y"],
    shuffle=True,
    filter_by=None
)

CPU times: user 6.27 ms, sys: 1.24 ms, total: 7.52 ms
Wall time: 10.6 ms


In [10]:
%%time
for i, batch in enumerate(vertex_loader):
    print("----Batch {}: Shape {}----".format(i, batch.shape))
    print(batch.head(1))

----Batch 0: Shape (131, 4)----
       vid    id                                                  x  y
0  5242900  1395  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...  0
----Batch 1: Shape (144, 4)----
       vid    id                                                  x  y
0  6291456  2292  0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 ...  4
----Batch 2: Shape (148, 4)----
       vid    id                                                  x  y
0  7340051  1935  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...  3
----Batch 3: Shape (141, 4)----
        vid    id                                                  x  y
0  14680101  1986  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...  3
----Batch 4: Shape (149, 4)----
   vid    id                                                  x  y
0   21  1824  0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...  5
----Batch 5: Shape (150, 4)----
        vid    id                                                  x  y
0  15728645  1459  0 0 0 0 0 0 0

#### For heterogeneous graphs

Since `Cora` is a homogeneous graph, we will connect to a different graph to demostrate the use case of heterogeneous graphs.

In [11]:
conn = TigerGraphConnection(
    host="http://127.0.0.1", # Change the address to your database server's
    graphname="hetero",
    username="tigergraph",
    password="tigergraph",
    gsqlSecret="" # secret instead of user/pass is required for TG cloud DBs created after 7/5/2022  
)

# Uncomment below to get and set token if token authentication is enabled. 
#conn.getToken(<secret>) # <secret> is your user secret. See https://docs.tigergraph.com/tigergraph-server/current/user-access/managing-credentials#_secrets for details.

In [12]:
print(conn.gsql("ls"))

---- Graph hetero
Vertex Types:
- VERTEX v0(PRIMARY_ID id INT, x LIST<DOUBLE>, y INT, train_mask BOOL, val_mask BOOL, test_mask BOOL) WITH STATS="OUTDEGREE_BY_EDGETYPE", PRIMARY_ID_AS_ATTRIBUTE="true"
- VERTEX v1(PRIMARY_ID id INT, x LIST<DOUBLE>, train_mask BOOL, val_mask BOOL, test_mask BOOL) WITH STATS="OUTDEGREE_BY_EDGETYPE", PRIMARY_ID_AS_ATTRIBUTE="true"
- VERTEX v2(PRIMARY_ID id INT, x LIST<DOUBLE>, train_mask BOOL, val_mask BOOL, test_mask BOOL) WITH STATS="OUTDEGREE_BY_EDGETYPE", PRIMARY_ID_AS_ATTRIBUTE="true"
Edge Types:
- DIRECTED EDGE v0v0(FROM v0, TO v0, is_train BOOL, is_val BOOL)
- DIRECTED EDGE v1v1(FROM v1, TO v1, is_train BOOL, is_val BOOL)
- DIRECTED EDGE v1v2(FROM v1, TO v2, is_train BOOL, is_val BOOL)
- DIRECTED EDGE v2v0(FROM v2, TO v0, is_train BOOL, is_val BOOL)
- DIRECTED EDGE v2v1(FROM v2, TO v1, is_train BOOL, is_val BOOL)
- DIRECTED EDGE v2v2(FROM v2, TO v2, is_train BOOL, is_val BOOL)

Graphs:
- Graph hetero(v0:v, v1:v, v2:v, v0v0:e, v1v1:e, v1v2:e, v2v0:e,

In [13]:
%%time
loader = conn.gds.vertexLoader(
    attributes={"v0": ["x", "y"],
                "v1": ["x"]},
    batch_size=20,
    shuffle=True,
    filter_by=None,
    loader_id=None,
    buffer_size=4,
)

Installing and optimizing queries. It might take a minute if this is the first time you use this loader.
Query installation finished.
CPU times: user 29.9 ms, sys: 4.49 ms, total: 34.4 ms
Wall time: 39.5 s


In [14]:
%%time
for i, batch in enumerate(loader):
    print("----Batch {}----".format(i))
    for j in batch:
        print("Vertex type:", j)
        print(batch[j].head(1))

----Batch 0----
Vertex type: v0
         vid                                                  x  y
0  163577857  0.386429 0.401847 -0.0926295 -0.0833778 0.6873...  2
Vertex type: v1
         vid                                                  x
0  178257921  -1.02871 -0.135028 -0.138423 -0.89996 1.68272 ...
----Batch 1----
Vertex type: v0
         vid                                                  x  y
0  142606336  -0.417304 -0.464672 -0.229524 0.336801 0.61027...  9
Vertex type: v1
         vid                                                  x
0  170917888  -0.0675672 -0.514478 1.91501 0.96472 -1.40837 ...
----Batch 2----
Vertex type: v0
         vid                                                  x  y
0  139460609  -0.846644 -0.686432 -2.26245 1.24922 0.576238 ...  8
Vertex type: v1
         vid                                                  x
0  197132289  -0.900331 -0.582497 -0.157183 -0.692183 0.7191...
----Batch 3----
Vertex type: v0
         vid                          

### Streaming through Kafka

**Note**: Kafka streaming function is only available for the Enterprise Edition. You need to activate the Enterprise Edition to use it. 

In [ ]:
conn = TigerGraphConnection(
    host="http://127.0.0.1", # Change the address to your database server's
    graphname="Cora",
    username="tigergraph",
    password="tigergraph",
    gsqlSecret="" # secret instead of user/pass is required for TG cloud DBs created after 7/5/2022  
)

# Uncomment below to get and set token if token authentication is enabled. 
#conn.getToken(<secret>) # <secret> is your user secret. See https://docs.tigergraph.com/tigergraph-server/current/user-access/managing-credentials#_secrets for details.

#### Configure Kafka
Set up Kafka here. Once configured, the settings will be shared with all newly created data loaders and no need to set up Kafka for each loader. Please see official [doc](https://docs.tigergraph.com/pytigergraph/current/gds/gds#_configurekafka) for detailed settings.

In [ ]:
conn.gds.configureKafka(
    kafka_address="127.0.0.1:9092",
    kafka_security_protocol="SASL_PLAINTEXT",
    kafka_sasl_mechanism="PLAIN",
    kafka_sasl_plain_username="your username",
    kafka_sasl_plain_password="your password"
)

#### Get batches of vertices

In [10]:
%%time
vertex_loader = conn.gds.vertexLoader(
    num_batches=20, 
    attributes=["id", "x", "y"],
    shuffle=True,
    filter_by=None,
)

Installing and optimizing queries. It might take a minute if this is the first time you use this loader.
Query installation finished.
CPU times: user 37.1 ms, sys: 11.9 ms, total: 48.9 ms
Wall time: 25.8 s


In [11]:
%%time
for i, batch in enumerate(vertex_loader):
    print("----Batch {}: Shape {}----".format(i, batch.shape))
    print(batch.head(1))

----Batch 0: Shape (157, 4)----
   vid    id                                                  x  y
0    9  1341  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...  0
----Batch 1: Shape (121, 4)----
       vid    id                                                  x  y
0  7340080  2332  0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 ...  0
----Batch 2: Shape (131, 4)----
        vid   id                                                  x  y
0  11534338  562  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...  2
----Batch 3: Shape (143, 4)----
        vid  id                                                  x  y
0  19922966  66  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 ...  4
----Batch 4: Shape (149, 4)----
       vid    id                                                  x  y
0  1048585  1443  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...  4
----Batch 5: Shape (125, 4)----
       vid    id                                                  x  y
0  8388625  2477  0 0 0 0 0 0 0 0 0 0